In [8]:
import numpy as np
import pyvista as pv
from kale.algorithms import contour_banded


In [9]:
# Requires PyVista 0.38.1
pv.set_jupyter_backend("server")
pv.global_theme.trame.interactive_ratio = 2
pv.global_theme.trame.still_ratio = 2
pv.global_theme.image_scale = 2  # upscales the saved screenshots/video frames

In [10]:
MESH_GEOMETRY_FILE_NAME = "nankai_121728_clean.vtk"
MESH_VALUES_FILE_NAME = "2023_03_04_00_02_03.hdf"
MESH_Z_SCALE = 0.025
TIME_STEP_INTERESTING = 1000
CONTOUR_LINE_WIDTH = 1
CONTOUR_LINE_COLOR = "black"
COLOR_MAPS = {
    "cumulative_slip": "CET_L19",  #'CET_C3',
    "geometric_moment": "CET_R3",
    "last_event_slip": "CET_L19",
    "loading_rate": "CET_C3",
}

COLORBAR_FONT_SIZE = 34
AXES_FONT_SIZE = 26
TIMESTEP_FONT_SIZE = 12

SCALAR_BAR_OPTS = dict(
    # Position (coordinates are on the viewport)
    vertical=True,
    height=0.20,
    width=0.05,
    position_x=0.05,
    position_y=0.40,
    # Labels
    title_font_size=AXES_FONT_SIZE,
    label_font_size=AXES_FONT_SIZE,
    n_labels=3,
    italic=False,
    fmt="%.1f",
    font_family="arial",
    shadow=True,
)

# Load Data

In [20]:
import xarray as xr
MESH_GEOMETRY_FILE_NAME = "nankai_121728_clean.vtk"
MESH_VALUES_FILE_NAME = "2023_03_04_00_02_03.hdf"
ds = xr.open_dataset(MESH_VALUES_FILE_NAME, engine="netcdf4")
data = ds["loading_rate"][0, :]
mesh = pv.read(MESH_GEOMETRY_FILE_NAME)
mesh.cell_data["data"] = data
mesh.points[:, -1] *= MESH_Z_SCALE

boundary = mesh.extract_feature_edges(
    boundary_edges=True,
    non_manifold_edges=False,
    feature_edges=False,
    manifold_edges=False,
)

SCALAR_BAR_OPTS = dict(
    vertical=True,
    height=0.00,
    width=0.00,
    position_x=0.05,
    position_y=0.40,
    title_font_size=0,
    label_font_size=0,
    n_labels=0,
    italic=False,
    fmt="%.1f",
    font_family="arial",
    shadow=False,
)


# Banded contour for a single time

In [23]:
pl = pv.Plotter()
pl.add_mesh(boundary, color=CONTOUR_LINE_COLOR, line_width=CONTOUR_LINE_WIDTH)
box = pv.Box(mesh.bounds)
pl.add_mesh(box, color="lightgrey", culling="front")
pl.camera.zoom(1.2)
pl.image_scale = 2

CONTOUR_LEVELS = np.linspace(-60, 60, 21)
N_COLORS = len(CONTOUR_LEVELS) - 1
CLIM = [np.min(CONTOUR_LEVELS), np.max(CONTOUR_LEVELS)]

contour, edges = contour_banded(
    mesh,
    CONTOUR_LEVELS,
    rng=CLIM,
    scalars="data",
)

pl.add_mesh(
    contour,
    cmap="RdYlBu_r",
    clim=CLIM,
    scalars="data",
    n_colors=N_COLORS,
    scalar_bar_args=dict(title=f"", **SCALAR_BAR_OPTS),
)
pl.add_mesh(
    edges.GetOutputPort(1), color=CONTOUR_LINE_COLOR, line_width=CONTOUR_LINE_WIDTH
)
pl.show()


Widget(value="<iframe src='http://localhost:52228/index.html?ui=P_0x29ea43b90_4&reconnect=auto' style='width: …